<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/Simple_Unet_Wetlands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rasterio scikit-image tensorflow keras gdown
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import glob
import gdown
import zipfile
import cv2
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import rasterio
import rasterio.plot
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import load_model
from keras.utils import Sequence, to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from skimage.util import random_noise
from scipy import ndimage
from scipy.ndimage import label as nd_label
from scipy.ndimage import generic_filter
from scipy.stats import mode


In [47]:
directory_path = '/content/data'
os.makedirs(directory_path, exist_ok=True)

In [48]:
csv_files = glob.glob('/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/*.csv')
csv_list = []
for csv_file in csv_files:
    base_name = os.path.basename(csv_file)[:-4]
    csv_dir = os.path.join(extract_dir, base_name)
    base_csv = pd.read_csv(csv_file)
    base_csv['subset'] = base_name
    csv_list.append(base_csv)
    index_csv = pd.concat(csv_list, ignore_index=True)

In [49]:
zip_files = glob.glob('/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/*.zip')
extract_dir = '/content/data' # destination directory
for zip_file in zip_files:
    base_name = os.path.basename(zip_file)[:-4]
    unzip_dir = os.path.join(extract_dir, base_name)
    os.makedirs(unzip_dir, exist_ok=True)
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(unzip_dir)

In [73]:
pause


NameError: ignored

In [50]:
unique_rows_df = index_csv.groupby(['tif_name', 'label', 'subset']).size().reset_index(name='Count')
unique_rows_df = index_csv.groupby(['tif_name', 'label', 'subset']).agg({'percent': 'mean'}).reset_index()

In [51]:
pivot_df = unique_rows_df.pivot(index=['tif_name', 'subset'], columns='label', values='percent').reset_index().fillna(0)

In [52]:
merged_df = index_csv.merge(pivot_df, on=['tif_name', 'subset'], how='left')
merged_df.drop(columns=['label'], inplace=True)

In [53]:
sum_df = merged_df.drop_duplicates(subset=['tif_name', 'subset'])

In [54]:
total_rows = len(sum_df)
train_fraction = 0.9 # modify this to set the training percentage
train_rows = int(total_rows * train_fraction)
val_rows = total_rows - train_rows

random_assignment = np.array([0] * train_rows + [1] * val_rows)
np.random.seed(42)
np.random.shuffle(random_assignment)
sum_df['random_split'] = random_assignment

<ipython-input-54-66bd3969d95d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['random_split'] = random_assignment


In [55]:
agg_df = sum_df.groupby(['random_split']).agg({13: 'mean',14:'mean',15: 'mean',16:'mean',17: 'mean',18:'mean'}).reset_index()
# agg_df = sum_df.groupby(['random_split']).agg({13: 'mean',14:'mean',15: 'mean'}).reset_index()
print(agg_df)

   random_split        13        14        15        16        17        18
0             0  0.152032  0.034274  0.011877  0.001600  0.001068  0.001292
1             1  0.145450  0.036990  0.009983  0.002517  0.001619  0.001482


In [56]:
# sum_df['Images_path']="/content/image_subsets/Maui_oneYear_Subset/Maui_Image_Subset/"+sum_df['tif_name']
# sum_df['Labels_path']="/content/image_subsets/Maui_oneYear_Subset/Maui_Label_Subset/"+sum_df['tif_name']


sum_df['Images_path']="/content/data/"+sum_df['subset']+'/Images/'+sum_df['tif_name']
sum_df['Labels_path']="/content/data/"+sum_df['subset']+'/Labels/'+sum_df['tif_name']

<ipython-input-56-0d7436cf7056>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['Images_path']="/content/data/"+sum_df['subset']+'/Images/'+sum_df['tif_name']
<ipython-input-56-0d7436cf7056>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['Labels_path']="/content/data/"+sum_df['subset']+'/Labels/'+sum_df['tif_name']


In [57]:
sum_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,percent,...,subset,13,14,15,16,17,18,random_split,Images_path,Labels_path
0,9217,1025,8705,513,61,438833.983073,440062.758637,2.457574e+06,2.458802e+06,0.000919,...,Kauai10_subset,0.000919,0.000000,0.000000,0.000000,0.000000,0.000000,0,/content/data/Kauai10_subset/Images/8705_513.tif,/content/data/Kauai10_subset/Labels/8705_513.tif
1,9729,1025,9217,513,62,440062.834396,441291.605845,2.457574e+06,2.458802e+06,0.033161,...,Kauai10_subset,0.033161,0.000000,0.000000,0.000000,0.000000,0.000000,1,/content/data/Kauai10_subset/Images/9217_513.tif,/content/data/Kauai10_subset/Labels/9217_513.tif
2,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,0.000130,...,Kauai10_subset,0.000130,0.000137,0.003502,0.000000,0.000000,0.000000,0,/content/data/Kauai10_subset/Images/13313_513.tif,/content/data/Kauai10_subset/Labels/13313_513.tif
5,15361,1025,14849,513,73,453579.566028,454808.362434,2.457574e+06,2.458802e+06,0.001663,...,Kauai10_subset,0.000000,0.000000,0.001663,0.000000,0.000000,0.000000,0,/content/data/Kauai10_subset/Images/14849_513.tif,/content/data/Kauai10_subset/Labels/14849_513.tif
6,9217,1537,8705,1025,104,438833.962924,440062.791964,2.456345e+06,2.457574e+06,0.007381,...,Kauai10_subset,0.007381,0.000000,0.000000,0.000000,0.000000,0.000000,0,/content/data/Kauai10_subset/Images/8705_1025.tif,/content/data/Kauai10_subset/Labels/8705_1025.tif
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6233,17921,15361,17409,14849,1079,308475.532176,309704.324054,2.156737e+06,2.157966e+06,0.012070,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000000,0.012070,0.000416,0.001926,0,/content/data/Hawaii_2010_008_subset/Images/17...,/content/data/Hawaii_2010_008_subset/Labels/17...
6236,3585,15873,3073,15361,1087,274069.129264,275297.911026,2.155509e+06,2.156737e+06,0.000507,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000507,0.000000,0.000000,0.000000,0,/content/data/Hawaii_2010_008_subset/Images/30...,/content/data/Hawaii_2010_008_subset/Labels/30...
6237,2049,16897,1537,16385,1156,270382.720297,271611.486128,2.153051e+06,2.154280e+06,0.000847,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000847,0.000000,0.000000,0.000000,0,/content/data/Hawaii_2010_008_subset/Images/15...,/content/data/Hawaii_2010_008_subset/Labels/15...
6238,17409,16897,16897,16385,1186,307246.720617,308475.512807,2.153051e+06,2.154280e+06,0.000889,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000000,0.000000,0.000000,0.000889,0,/content/data/Hawaii_2010_008_subset/Images/16...,/content/data/Hawaii_2010_008_subset/Labels/16...


In [58]:
# example to show the width, height and bands of the images
def get_image_shapes_in_folders(folder_paths):
    image_shapes = []
    for folder_path in folder_paths:
        for root, dirs, files in os.walk(folder_path):
            # Sort the files alphabetically
            files = sorted(files)
            for file in files:
                if file.endswith('.tif') or file.endswith('.jpg') or file.endswith('.png'):
                    image_path = os.path.join(root, file)
                    try:
                        with rasterio.open(image_path) as src:
                            width, height = src.width, src.height
                            band_count = src.count  # Number of bands in the image
                            image_shapes.append((file, width, height, band_count))
                    except Exception as e:
                        print(f"Error getting shape of image '{file}': {e}")
    return image_shapes

folder_paths = ["/content/data/Kauai_subset/Images"]

shapes = get_image_shapes_in_folders(folder_paths)
for shape in shapes:
    file, width, height, band_count = shape
    print(f" {file[:-4]}: {width}, {height}, {band_count}")

 10241_1025: 512, 512, 7
 10241_1537: 512, 512, 7
 10241_2049: 512, 512, 7
 10241_2561: 512, 512, 7
 10241_5633: 512, 512, 7
 10241_6145: 512, 512, 7
 10241_6657: 512, 512, 7
 10241_7169: 512, 512, 7
 10241_7681: 512, 512, 7
 10241_8193: 512, 512, 7
 10753_1025: 512, 512, 7
 10753_14849: 512, 512, 7
 10753_1537: 512, 512, 7
 10753_15873: 512, 512, 7
 10753_2049: 512, 512, 7
 10753_2561: 512, 512, 7
 10753_3073: 512, 512, 7
 10753_3585: 512, 512, 7
 10753_4609: 512, 512, 7
 10753_6145: 512, 512, 7
 10753_6657: 512, 512, 7
 10753_7169: 512, 512, 7
 10753_7681: 512, 512, 7
 11265_1025: 512, 512, 7
 11265_14337: 512, 512, 7
 11265_1537: 512, 512, 7
 11265_3585: 512, 512, 7
 11265_4097: 512, 512, 7
 11265_6657: 512, 512, 7
 11265_7169: 512, 512, 7
 11265_7681: 512, 512, 7
 11777_12289: 512, 512, 7
 11777_12801: 512, 512, 7
 11777_1537: 512, 512, 7
 11777_16385: 512, 512, 7
 11777_2049: 512, 512, 7
 11777_4097: 512, 512, 7
 11777_4609: 512, 512, 7
 11777_5121: 512, 512, 7
 11777_7169: 512, 5

In [59]:
def load_and_reshape_image(image_path, img_height, img_width):
    with rasterio.open(image_path) as src:
        image = src.read()
        return image

In [60]:
def load_images_and_labels(image_files, label_files, img_height, img_width, num_classes):
  images = []
  labels = []

  # image_files = glob.glob(os.path.join(image_files, "*.tif"))
  for image_file in image_files:
      image = load_and_reshape_image(image_file, img_height, img_width)
      images.append(image)

  # label_files = glob.glob(os.path.join(label_files, "*.tif"))
  for label_file in label_files:
      label = load_and_reshape_image(label_file, img_height, img_width)
      label -= 1  # adjust labels to be in the range 0-8 instead of 1-9
      label = to_categorical(label, num_classes=num_classes)   # one-hot encode the labels
      labels.append(label)

  return np.array(images), np.array(labels)

In [61]:
class DataGenerator(Sequence):
    def __init__(self, image_files, label_files, img_height, img_width, batch_size, num_classes):
        self.image_files = image_files
        self.label_files = label_files
        self.img_height = img_height
        self.img_width = img_width
        self.batch_size = batch_size
        self.num_classes = num_classes
        # self.noise = noise

    def __len__(self):
        return int(np.ceil(len(self.image_files) / self.batch_size))

    def __getitem__(self, index):
        batch_files = self.image_files[index * self.batch_size : (index + 1) * self.batch_size]
        batch_images, batch_labels = self.load_images_and_labels(batch_files)
        return batch_images, batch_labels

    def load_and_reshape_image(self, image_path):
        with rasterio.open(image_path) as src:
            image = src.read()
            image = image.transpose((1, 2, 0))
            if image.shape[0] != self.img_height or image.shape[1] != self.img_width:
                image = cv2.resize(image, (self.img_width, self.img_height), interpolation=cv2.INTER_NEAREST)
            if len(image.shape) == 3 and image.shape[2] == 1:
                image = np.squeeze(image, axis=2)

            return image

    def load_images_and_labels(self, image_files):
        images = []
        labels = []

        for image_file in image_files:
            image = self.load_and_reshape_image(image_file)
            image[image <= -3e+38] = np.nan

            # # Replace NaN values with the mean of the non-NaN pixels
            if np.any(np.isnan(image)):
                nan_mask = np.isnan(image)
                image[nan_mask] = np.nanmean(image)

            # # Replace Inf values with the mean of the non-Inf pixels
            if np.any(np.isinf(image)):
                inf_mask = np.isinf(image)
                image[inf_mask] = np.nanmean(image)

            # Convert to float
            image = image.astype(np.float32)

            # # Z-score normalization
            # mean = np.mean(image, axis=(0, 1), keepdims=True)
            # std = np.std(image, axis=(0, 1), keepdims=True)
            # # mean[mean < 0]
            # std[std < 0] = 0

            # # Normalize with epsilon to prevent divide by zero
            # epsilon = 1e-7

            # image = (image - mean) / (std + epsilon)

            images.append(image)

        for image_file in image_files:
            label_file = image_file.replace("Images", "Labels")
            label = self.load_and_reshape_image(label_file)
            label -= 1
            label = to_categorical(label, num_classes=self.num_classes)
            labels.append(label)

        return np.array(images), np.array(labels)

In [62]:
# import TenserFlow classes and functions
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization

In [63]:
def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('LeakyReLU')(encoder) # from relu to LeakyReLU
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('LeakyReLU')(encoder) # from relu to LeakyReLU
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('LeakyReLU')(decoder) # from relu to LeakyReLU
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('LeakyReLU')(decoder) # from relu to LeakyReLU
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('LeakyReLU')(decoder) # from relu to LeakyReLU
	return decoder

def get_model():
	inputs = layers.Input(shape=[KERNEL_SIZE, KERNEL_SIZE, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = layers.Conv2D(25, (1, 1), activation='softmax')(decoder0) # kept because this is a multiclass classification

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER),
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

	return model

In [64]:
def create_model(img_size, num_classes):
    inputs = keras.Input(shape=(img_size[0], img_size[1], 4))  # Change the number of channels to 4

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("LeakyReLU")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("LeakyReLU")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("LeakyReLU")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("LeakyReLU")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("LeakyReLU")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])  # updated to sparse categorical cross-entropy loss
    return model

In [65]:
def predict_and_save_segments(input_folder, output_folder, model, img_height, img_width):
      # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Get a list of input files
    input_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]

    for filename in input_files:
              # Read input image
        image_path = os.path.join(input_folder, filename)

        with rasterio.open(image_path) as src:
            # Read image data and reshape
            image = src.read()  # Read all bands

        image = load_and_reshape_image(image_path,img_height, img_width)
        image = image.astype(np.uint8)
        # print(image.shape)
        masks = mask_generator.generate(image)

        flat_mask = show_anns(masks)
        imagery_file = rasterio.open(image_path)
        imagery_transform = imagery_file.transform
        reshaped_image = rasterio.plot.reshape_as_raster(flat_mask)
        reshaped_image = reshaped_image[0]
        # Get metadata from the input image
        # print(reshaped_image.shape)
        meta = src.meta

        # Update metadata for the output image
        meta.update(count=1, dtype=reshaped_image.dtype)

        # Create output path
        output_path = os.path.join(output_folder, filename)

        # Write all 9 prediction channels as separate bands
        with rasterio.open(output_path, 'w', **meta) as dst:
            # for i in range(9):
            dst.write(reshaped_image,1)  # Write each channel as a separate band

        print(f"Saved prediction for {filename}")

    print("Prediction and saving completed.")

In [66]:
def predict_and_save(input_folder, output_folder, model, img_height, img_width):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Get a list of input files
    input_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]

    for filename in input_files:
        # Read input image
        input_path = os.path.join(input_folder, filename)
        with rasterio.open(input_path) as src:
            # Read image data and reshape
            image = src.read()  # Read all bands

            image[image <= -3e+38] = np.nan

            # # Replace NaN values with the mean of the non-NaN pixels
            if np.any(np.isnan(image)):
                nan_mask = np.isnan(image)
                image[nan_mask] = np.nanmean(image)

            # # Replace Inf values with the mean of the non-Inf pixels
            if np.any(np.isinf(image)):
                inf_mask = np.isinf(image)
                image[inf_mask] = np.nanmean(image)

            # Convert to float
            image = image.astype(np.float32)

            # # Z-score normalization
            # mean = np.mean(image, axis=(0, 1), keepdims=True)
            # std = np.std(image, axis=(0, 1), keepdims=True)
            # # mean[mean < 0]
            # std[std < 0] = 0

            # # Normalize with epsilon to prevent divide by zero
            # epsilon = 1e-7

            # image = (image - mean) / (std + epsilon)

            image = np.transpose(image, (1, 2, 0))  # Transpose to (height, width, bands)
            image = cv2.resize(image, (img_width, img_height), interpolation=cv2.INTER_NEAREST)
            image = np.expand_dims(image, axis=0)  # Add batch dimension
            # # Perform prediction
            prediction = model.predict(image)
            prediction[prediction <= 0] = np.nan
            prediction = prediction*255
            prediction = prediction.astype(np.uint8)

        # Get metadata from the input image
        meta = src.meta

        # Update metadata for the output image
        meta.update(count=26, dtype=prediction.dtype,nodata = 0)
        # meta.
        # Create output path
        output_path = os.path.join(output_folder, filename)

        # Write all 9 prediction channels as separate bands
        with rasterio.open(output_path, 'w', **meta) as dst:
            for i in range(25):
                dst.write(prediction[0, :, :, i], i + 1)  # Write each channel as a separate band

            # Add a 10th band containing the argmax of the 9 channels
            argmax_band = np.argmax(prediction[0], axis=-1)
            dst.write(argmax_band, 26)

        print(f"Saved prediction for {filename}")

    print("Prediction and saving completed.")

In [67]:
KERNEL_SIZE = 512
BANDS = range(7)
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
OPTIMIZER = 'adam'
LOSS = 'categorical_crossentropy'
METRICS = ['categorical_accuracy']

In [68]:
model = get_model()
print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 512, 512, 7)]        0         []                            
                                                                                                  
 conv2d_46 (Conv2D)          (None, 512, 512, 32)         2048      ['input_3[0][0]']             
                                                                                                  
 batch_normalization_54 (Ba  (None, 512, 512, 32)         128       ['conv2d_46[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_54 (Activation)  (None, 512, 512, 32)         0         ['batch_normalization_54

In [69]:
# Create the data generator
training_data_generator = DataGenerator(sum_df['Images_path'][sum_df['random_split']==0], sum_df['Labels_path'][sum_df['random_split']==0], 512, 512, 4, 25)
validation_data_generator = DataGenerator(sum_df['Images_path'][sum_df['random_split']==1], sum_df['Labels_path'][sum_df['random_split']==1], 512, 512, 4, 25)

In [70]:
batch_images, batch_labels = training_data_generator.__getitem__(0)

In [71]:
batch_images.shape

(4, 512, 512, 7)

In [74]:
batch_labels.shape

(4, 512, 512, 25)

In [72]:
batch_labels.shape# Train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','categorical_accuracy'])
callbacks = [
    keras.callbacks.ModelCheckpoint("landcover_segmentation.h5", save_best_only=True)
]

model.fit(training_data_generator, validation_data=validation_data_generator, epochs=5, callbacks=callbacks,shuffle=True)
# model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=16, epochs=10)

Epoch 1/5
730/730 [==============================] - ETA: 0s - loss: 1.9545 - accuracy: 0.4089 - categorical_accuracy: 0.4089

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


730/730 [==============================] - 751s 973ms/step - loss: 1.9545 - accuracy: 0.4089 - categorical_accuracy: 0.4089 - val_loss: 1.5497 - val_accuracy: 0.4758 - val_categorical_accuracy: 0.4758
Epoch 2/5
730/730 [==============================] - 701s 959ms/step - loss: 1.6596 - accuracy: 0.4569 - categorical_accuracy: 0.4569 - val_loss: 1.5188 - val_accuracy: 0.4828 - val_categorical_accuracy: 0.4828
Epoch 3/5
730/730 [==============================] - 694s 950ms/step - loss: 1.5851 - accuracy: 0.4758 - categorical_accuracy: 0.4758 - val_loss: 1.6075 - val_accuracy: 0.4421 - val_categorical_accuracy: 0.4421
Epoch 4/5
730/730 [==============================] - 676s 925ms/step - loss: 1.5315 - accuracy: 0.4906 - categorical_accuracy: 0.4906 - val_loss: 1.6453 - val_accuracy: 0.4546 - val_categorical_accuracy: 0.4546
Epoch 5/5
730/730 [==============================] - 712s 976ms/step - loss: 1.4899 - accuracy: 0.5001 - categorical_accuracy: 0.5001 - val_loss: 1.5275 - val_accurac

In [76]:
# loss, accuracy = model.evaluate(validation_data_generator)
# print("Validation Loss:", loss)
# print("Validation Accuracy:", accuracy)
evaluation_results = model.evaluate(validation_data_generator)
print("Evaluation Results:", evaluation_results)



82/82 [==============================] - 60s 722ms/step - loss: 1.5275 - accuracy: 0.5030 - categorical_accuracy: 0.5030
Evaluation Results: [1.527496337890625, 0.502964973449707, 0.502964973449707]


In [77]:
predict_and_save('/content/data/Maui10_subset/Images', '/content/data/Maui10_subset/Predictions', model, 512, 512)

1/1 [==============================] - 2s 2s/step
Saved prediction for 24577_14849.tif
1/1 [==============================] - 0s 29ms/step
Saved prediction for 25601_13825.tif
1/1 [==============================] - 0s 26ms/step
Saved prediction for 8193_3585.tif
1/1 [==============================] - 0s 28ms/step
Saved prediction for 19457_7169.tif
1/1 [==============================] - 0s 26ms/step
Saved prediction for 5633_3073.tif
1/1 [==============================] - 0s 30ms/step
Saved prediction for 18945_6657.tif
1/1 [==============================] - 0s 27ms/step
Saved prediction for 18433_6657.tif
1/1 [==============================] - 0s 25ms/step
Saved prediction for 12801_7681.tif
1/1 [==============================] - 0s 28ms/step
Saved prediction for 28673_12801.tif
1/1 [==============================] - 0s 25ms/step
Saved prediction for 8193_7169.tif
1/1 [==============================] - 0s 25ms/step
Saved prediction for 24065_11777.tif
1/1 [============================